# Morphology (Tutorial 3)
***
# Table of Contents
1.   [Setup](#Setup)
2.   [Exercise 1 - Connected Components](#Exercise-1---Connected-Components)
3.   [Exercise 2 - Dilation](#Exercise-2---Dilation)
4.   [Exercise 3 - Erosion](#Exercise-3---Erosion)
5.   [Exercise 4 - Opening](#Exercise-4---Opening)
6.   [Exercise 5 - Closing](#Exercise-5---Closing)
7.   [Exercise 6 - Line Segmentation](#Exercise-6---Line-Segmentation)

# Setup

These libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms
* os - File traversal
* tqdm.notebook - tqdm progress bars, but for ipynb files
* Classes - Custom classes written by me for this assignment

In [1]:
%%capture

import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm

# A slightly modified generateHistograms function from tutorial 1

def generateHistogram(image, title):
    hist = cv2.calcHist(image, [0], None, [255], [0, 255])
    plt.ylabel('Pixels')
    plt.xlabel('Intensity')
    plt.title(title)
    plt.plot(hist)
    plt.xlim([0, 256])
    plt.savefig('Histograms/' + title + ' histogram.png')
    plt.clf()


shapes = cv2.imread('Images/shapes.jpg')
euro = cv2.imread('Images/Euro_Coins.jpg')
text = cv2.imread('Images/text.png')

# Exercise 1 - Connected Components

In [2]:
def getObjects(image, title):
    # gray scale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # blur using gaussian to help reduce noise
    dst = cv2.GaussianBlur(gray,(13,13),cv2.BORDER_DEFAULT)
    # treshold image
    ret, thresh = cv2.threshold(dst,230,255,cv2.THRESH_BINARY_INV)
    # save treshold
    cv2.imwrite("Output/treshold "+title+".png", thresh, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # get labels
    ret, labels = cv2.connectedComponents(thresh)

    # Merge image back together
    label_hue = np.uint8(179 * labels / np.max(labels))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue == 0] = 0
    # save labels as image
    cv2.imwrite("Output/"+title+str(ret-1)+".png", labeled_img, [cv2.IMWRITE_PNG_COMPRESSION, 0])

getObjects(shapes.copy(), "shapes")
getObjects(euro.copy(), "euro")

In [3]:
def getROI(image, title):
    # gray scale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # blur using gaussian to help reduce noise
    dst = cv2.GaussianBlur(gray,(13,13),cv2.BORDER_DEFAULT)
    # treshold image
    ret, thresh = cv2.threshold(dst,230,255,cv2.THRESH_BINARY_INV)
    # getting ROIs with findContours
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    i = 1
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        ROI = image[y:y+h,x:x+w]
        cv2.imwrite("Output/"+title+" object "+str(i)+".png", ROI, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        i+=1

getROI(shapes.copy(), "shapes")
getROI(euro.copy(), "euro")

# Exercise 2 - Dilation

In [4]:
%%capture
kernel = np.ones((3, 3), 'uint8')
dilate = cv2.dilate(text.copy(), kernel)
cv2.imwrite("Output/dilated text 3x3.png", dilate, [cv2.IMWRITE_PNG_COMPRESSION, 0])
kernel = np.ones((5, 5), 'uint8')
dilate = cv2.dilate(text.copy(), kernel)
cv2.imwrite("Output/dilated text 5x5.png", dilate, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# Exercise 3 - Erosion

In [5]:
%%capture
kernel = np.ones((3, 3), 'uint8')
erode = cv2.erode(text.copy(), kernel)
cv2.imwrite("Output/eroded text 3x3.png", erode, [cv2.IMWRITE_PNG_COMPRESSION, 0])
kernel = np.ones((5, 5), 'uint8')
erode = cv2.erode(text.copy(), kernel)
cv2.imwrite("Output/eroded text 5x5.png", erode, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# Exercise 4 - Opening

In [6]:
%%capture
kernel = np.ones((3, 3), 'uint8')
open = cv2.morphologyEx(text.copy(), cv2.MORPH_OPEN ,kernel)
cv2.imwrite("Output/open text 3x3.png", open, [cv2.IMWRITE_PNG_COMPRESSION, 0])
kernel = np.ones((5, 5), 'uint8')
open = cv2.morphologyEx(text.copy(), cv2.MORPH_OPEN ,kernel)
cv2.imwrite("Output/open text 5x5.png", open, [cv2.IMWRITE_PNG_COMPRESSION, 0])


# Exercise 5 - Closing

In [7]:
%%capture
kernel = np.ones((3, 3), 'uint8')
close = cv2.morphologyEx(text.copy(), cv2.MORPH_CLOSE ,kernel)
cv2.imwrite("Output/close text 3x3.png", close, [cv2.IMWRITE_PNG_COMPRESSION, 0])
kernel = np.ones((5, 5), 'uint8')
close = cv2.morphologyEx(text.copy(), cv2.MORPH_CLOSE ,kernel)
cv2.imwrite("Output/close text 5x5.png", close, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# Exercise 6 - Line Segmentation

In [87]:
gray = cv2.cvtColor(text.copy(),cv2.COLOR_BGR2GRAY)
dst = cv2.GaussianBlur(gray, (13,13), cv2.BORDER_DEFAULT)
ret, thresh = cv2.threshold(dst, 187, 255, cv2.THRESH_BINARY)
kernel = np.ones((3, 24), 'uint8')
dilate = cv2.morphologyEx(thresh, cv2.MORPH_DILATE ,kernel)
kernel = np.ones((3, 400), 'uint8')
erode = cv2.morphologyEx(dilate, cv2.MORPH_ERODE ,kernel)
final = cv2.bitwise_not(erode)
cv2.imwrite("test 0.png", final, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# get labels
ret, labels = cv2.connectedComponents(final)

# Merge image back together
label_hue = np.uint8(179 * labels / np.max(labels))
blank_ch = 255 * np.ones_like(label_hue)
labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
labeled_img[label_hue == 0] = 0
# save labels as image
cv2.imwrite("test 3.png", labeled_img, [cv2.IMWRITE_PNG_COMPRESSION, 0])

gray = cv2.cvtColor(text.copy(),cv2.COLOR_BGR2GRAY)
kernel = np.ones((3, 3), 'uint8')
close = cv2.morphologyEx(gray, cv2.MORPH_CLOSE ,kernel)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
im = cv2.filter2D(close, -1, kernel)
kernel = np.ones((2, 2), 'uint8')
erode = cv2.morphologyEx(im, cv2.MORPH_ERODE ,kernel)
ret, thresh = cv2.threshold(erode, 0, 255, cv2.THRESH_BINARY)
txt = cv2.bitwise_not(thresh)
cv2.imwrite("test 4.png", thresh, [cv2.IMWRITE_PNG_COMPRESSION, 0])


c_text = text.copy()
for i, x in enumerate(labeled_img):
    for j, y in enumerate(x):
        # print(np.sum(txt[i][j]))
        if np.sum(txt[i][j]) == 255:
            c_text[i][j] = y

cv2.imwrite("test 5.png", c_text, [cv2.IMWRITE_PNG_COMPRESSION, 0])


True